In [25]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait


In [36]:
#Scraping NBC news wild fires with Selenium and Beautifulsoup
def get_list_link_from_subject(subject, num_pages):


    PATH = "./chromedriver_win32/chromedriver.exe"
    driver = webdriver.Chrome(PATH)

    link_list = []

    driver.get("https://www.google.com/search?q="+subject+"&rlz=1C1CHBF_frFR863FR863&biw=1920&bih=880&sxsrf=AOaemvI0XcPZB9YWw9GUVGwWTEXPDVqRxQ:1638967714934&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjGlsnDntT0AhWTTcAKHeyuDk4Q_AUoAXoECAEQAw")

    driver.find_element(By.XPATH, "//button[@class='VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-k8QpJ VfPpkd-LgbsSe-OWXEXe-dgl2Hf nCP5yc AjY5Oe DuMIQc']").click() #accept google policy

    for i in range(num_pages):
        if i != 0:
            driver.find_element(By.ID, "pnnext").click()

        html_source = driver.page_source

        soup = BeautifulSoup(html_source, 'lxml')

        #Getting all g-card 
        g_card_list = soup.find_all("g-card")

        for g_card in g_card_list:
            a = g_card.find("a")
            link = a['href']
            link_list.append(link)

    driver.quit()

    print("Successfully scraped : ", len(link_list), " links")

    return link_list



In [37]:
#Now that we have the link list, for each link try to scrape the article if there is one and the date if there is one.
def get_df_from_link_list(link_list):

    my_timeout = 10

    data = []

    for i, link in enumerate(link_list):
        d = {}
        print(i+1, "/", len(link_list))

        try:
            html_text = requests.get(link, timeout=my_timeout).text

            soup = BeautifulSoup(html_text, 'lxml')

            title = soup.find('title')
            if title != None:
                d["Title"] = title.text

            d["Link"] = link
   
            time = soup.find('time')
            if time != None:
                if time.has_attr('datetime'):

                    d["Date"] = time['datetime']
            
            article = soup.find('article')
            if article != None:
                paragraphs = article.find_all('p')
            big_p = ""
            for p in paragraphs:
                big_p = big_p + " " + p.text 
            
            if big_p != "":
                d["Content"] = big_p
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as err: #Requests takes way too long
            print('Server taking too long. Try again later for page number' + str(i))

        data.append(d)

    return pd.DataFrame(data)

In [38]:
def get_df_from_subject(subject,num_pages):
    return get_df_from_link_list(get_list_link_from_subject(subject, num_pages))

In [39]:
my_scraped_data = get_df_from_subject("pokemon",5)

C:\Users\SORLUC~1\AppData\Local\Temp/ipykernel_28448/477364689.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


Successfully scraped :  49  links
1 / 49
yo1
yo2
yo3


In [ ]:
my_scraped_data

In [23]:
gnews_df.isna().sum()

Title      0
Link       0
Date       0
Content    0
dtype: int64